In [2]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import numpy as np
import pandas as pd

In [3]:
# 产生随机分类数据集，10个特征， 2个类别
x, y = make_classification(n_samples=1000, n_features=5, n_classes=2)

In [8]:
# 步骤一：构造黑盒目标函数
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
                               min_samples_split=int(min_samples_split),
                               max_features=min(max_features, 0.999),  # float
                               max_depth=int(max_depth),
                               random_state=2),
        x, y, scoring='f1', cv=5
    ).mean()
    return val

In [9]:
# 步骤二：确定取值空间
pbounds = {'n_estimators': (10, 250),  # 表示取值范围为10至250
           'min_samples_split': (2, 25),
           'max_features': (0.1, 0.999),
           'max_depth': (5, 15)}

In [10]:
# 步骤三：构造贝叶斯优化器
optimizer = BayesianOptimization(
    f=rf_cv,  # 黑盒目标函数
    pbounds=pbounds,  # 取值空间
    verbose=2,  # verbose = 2 时打印全部，verbose = 1 时打印运行中发现的最大值，verbose = 0 将什么都不打印
    random_state=1,
)

In [11]:
optimizer.maximize(  # 运行
    init_points=5,  # 随机搜索的步数
    n_iter=25,  # 执行贝叶斯优化迭代次数
)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9494   |  9.17     |  0.7476   |  2.003    |  82.56    |
|  2        |  0.9451   |  6.468    |  0.183    |  6.284    |  92.93    |
|  3        |  0.9485   |  8.968    |  0.5844   |  11.64    |  174.5    |
|  4        |  0.9493   |  7.045    |  0.8894   |  2.63     |  170.9    |
|  5        |  0.9494   |  9.173    |  0.6023   |  5.229    |  57.54    |
|  6        |  0.9444   |  10.36    |  0.1358   |  5.1      |  56.42    |
|  7        |  0.9485   |  9.426    |  0.4717   |  14.6     |  188.6    |
|  8        |  0.9485   |  9.883    |  0.3823   |  14.96    |  188.7    |
|  9        |  0.9525   |  8.625    |  0.7956   |  2.543    |  83.2     |
|  10       |  0.9495   |  9.966    |  0.3076   |  3.452    |  83.79    |
|  11       |  0.9525   |  8.406    |  0.6192   |  2.842    |  83.96    |
|  12       |  0.9515   |  7.114    | 

In [12]:
print(optimizer.res)  # 打印所有优化的结果
print(optimizer.max)  # 最好的结果与对应的参数

[{'target': 0.9494347999396158, 'params': {'max_depth': 9.17022004702574, 'max_features': 0.7475717196045001, 'min_samples_split': 2.002630620798932, 'n_estimators': 82.55981743164155}}, {'target': 0.94506308257516, 'params': {'max_depth': 6.467558908171131, 'max_features': 0.18301239669714925, 'min_samples_split': 6.283984861686431, 'n_estimators': 92.93457449033146}}, {'target': 0.9484974472026451, 'params': {'max_depth': 8.967674742306698, 'max_features': 0.5843962438690179, 'min_samples_split': 11.64147383127578, 'n_estimators': 174.4526800952223}}, {'target': 0.9493391899928312, 'params': {'max_depth': 7.0445224973151745, 'max_features': 0.8894275753154599, 'min_samples_split': 2.6299146435523015, 'n_estimators': 170.91220244281652}}, {'target': 0.9494037023760488, 'params': {'max_depth': 9.17304802367127, 'max_features': 0.6022621557727307, 'min_samples_split': 5.2288995876903765, 'n_estimators': 57.54435738037091}}, {'target': 0.9444328066722832, 'params': {'max_depth': 10.36008